# **BASE**

In [2]:
%pip install tabulate tk

Note: you may need to restart the kernel to use updated packages.


In [1]:
from operator import itemgetter
from tabulate import tabulate
import shutil
from datetime import datetime, timedelta
import os
from collections import Counter
import tkinter as tk
from tkinter import messagebox

In [11]:

# Placeholder for the tasks
tasks = []
# Archive list for completed tasks
archived_tasks = []

# List to store completed tasks history
completed_tasks_history = []

# Function to generate task analytics
def generate_task_analytics():
    total_tasks = len(tasks)
    completed_tasks = sum(1 for task in tasks if task['status'] == 'Completed')
    pending_tasks = total_tasks - completed_tasks

    # Count tasks by priority
    priority_count = Counter(task['priority'] for task in tasks)

    # Count tasks by category
    category_count = Counter(task['category'] for task in tasks)

    # Display the analytics
    print("\n--- Task Analytics ---")
    print(f"Total Tasks: {total_tasks}")
    print(f"Completed Tasks: {completed_tasks} ({(completed_tasks / total_tasks) * 100:.2f}%)")
    print(f"Pending Tasks: {pending_tasks} ({(pending_tasks / total_tasks) * 100:.2f}%)")

    print("\nTasks by Priority:")
    for priority, count in priority_count.items():
        print(f"{priority}: {count} task(s)")

    print("\nTasks by Category:")
    for category, count in category_count.items():
        print(f"{category}: {count} task(s)")

    print("\n--- Upcoming Due Dates ---")
    # Sort tasks by due date
    sorted_due_dates = sorted(tasks, key=lambda x: x['due_date'])
    for task in sorted_due_dates:
        print(f"Task '{task['name']}' (Due: {task['due_date']})")

# Function to mark a task as completed and add it to the completed task history
def mark_task_completed(index):
    if index < 0 or index >= len(tasks):
        print("Invalid task number.")
        return

    task = tasks.pop(index)
    task['status'] = 'Completed'
    task['completed_on'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Add completion date
    completed_tasks_history.append(task)
    print(f"Task '{task['name']}' has been marked as completed.")

# Function to view completed tasks history
def view_completed_tasks_history():
    if not completed_tasks_history:
        print("No tasks have been completed yet.")
    else:
        print("Completed Tasks History:")
        for i, task in enumerate(completed_tasks_history, start=1):
            print(f"{i}. Task: {task['name']}, Due Date: {task['due_date']}, Priority: {task['priority']}, Completed on: {task['completed_on']}")

# Function to set a reminder for a task
def set_task_reminder(index, reminder_days_before):
    if index < 0 or index >= len(tasks):
        print("Invalid task number.")
        return

    task = tasks[index]
    due_date = datetime.strptime(task['due_date'], "%Y-%m-%d")
    reminder_date = due_date - timedelta(days=reminder_days_before)

    task['reminder'] = reminder_date.strftime("%Y-%m-%d")
    print(f"Reminder set for task '{task['name']}' on {task['reminder']}.")

# Function to check and notify about tasks approaching their due date
def check_due_dates():
    today = datetime.now()
    upcoming_tasks = []

    for task in tasks:
        due_date = datetime.strptime(task['due_date'], "%Y-%m-%d")
        if due_date <= today + timedelta(days=1) and task.get('status') != 'Completed':
            upcoming_tasks.append(task)

    if upcoming_tasks:
        print("Upcoming Tasks (Due within the next 24 hours):")
        for task in upcoming_tasks:
            print(f"Task: {task['name']}, Due Date: {task['due_date']}, Priority: {task['priority']}")
    else:
        print("No tasks approaching their due date in the next 24 hours.")

# Function to archive completed tasks
def archive_completed_tasks():
    global archived_tasks
    completed_tasks = [task for task in tasks if task.get('status') == 'Completed']
    archived_tasks.extend(completed_tasks)
    tasks[:] = [task for task in tasks if task.get('status') != 'Completed']
    print(f"{len(completed_tasks)} tasks archived.")

# Function to create a backup of the tasks file
def backup_tasks_file():
    if not os.path.exists('tasks.json'):
        print("No tasks file found to backup.")
        return

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_file_name = f"tasks_backup_{timestamp}.json"
    shutil.copy('tasks.json', backup_file_name)
    print(f"Backup created: {backup_file_name}")

# Input validation for dates
def is_valid_date(date_str):
    try:
        datetime.strptime(date_str, "%Y-%m-%d")
        return True
    except ValueError:
        return False

# Function to display the menu
def display_menu():
    print("\nTask Management System")
    print("1. Add a Task")
    print("2. View All Tasks")
    print("3. Edit a Task")
    print("4. Delete a Task")
    print("5. Exit")
    print("6. Sort Tasks by Due Date")
    print("7. Sort Tasks by Priority")
    print("8. Filter Tasks by Category")
    print("9. Mark Task as Completed")
    print("10. Archive Completed Tasks")
    print("11. Check Due Dates (Reminder)")
    print("12. Set Task Reminder")
    print("13. View Completed Tasks History")  # New option for viewing completed tasks
    print("14. Generate Task Analytics")  # New option for analytics
    print("15. Exit")

# Updated add_task function with validation
def add_task(task_name, due_date, priority, category):
    if not task_name.strip():
        print("Task name cannot be empty!")
        return
    if not is_valid_date(due_date):
        print("Invalid date format! Use YYYY-MM-DD.")
        return
    if priority not in ["High", "Medium", "Low", 'high', 'medium', 'low']:
        print("Invalid priority! Choose from High, Medium, or Low.")
        return

    task = {
        'name': task_name,
        'due_date': due_date,
        'priority': priority,
        'category': category,
        'status': 'Pending',  # default status
    }
    tasks.append(task)
    print(f"Task '{task_name}' added successfully!")

# Function to view tasks including their completion status
def view_tasks():
    if not tasks:
        print("No tasks available.")
    else:
        headers = ["#", "Task Name", "Due Date", "Priority", "Category", "Status"]
        table = [[i + 1, task['name'], task['due_date'], task['priority'], task['category'], task.get('status', 'Incomplete')] for i, task in enumerate(tasks)]
        print(tabulate(table, headers=headers, tablefmt="grid"))



# Function to edit a task
def edit_task(index, new_name, new_due_date, new_priority, new_category):
    if 0 <= index < len(tasks):
        tasks[index]['name'] = new_name
        tasks[index]['due_date'] = new_due_date
        tasks[index]['priority'] = new_priority
        tasks[index]['category'] = new_category
        print(f"Task {index + 1} updated successfully!")
    else:
        print("Invalid task index!")

# Function to delete a task
def delete_task(index):
    if 0 <= index < len(tasks):
        removed_task = tasks.pop(index)
        print(f"Task '{removed_task['name']}' deleted successfully!")
    else:
        print("Invalid task index!")

# Update sorting and filtering functions to use tabulate
def sort_tasks(by):
    if not tasks:
        print("No tasks to sort.")
        return
    if by == "due_date":
        sorted_tasks = sorted(tasks, key=itemgetter('due_date'))
    elif by == "priority":
        priority_order = {"High": 1, "Medium": 2, "Low": 3}
        sorted_tasks = sorted(tasks, key=lambda x: priority_order.get(x['priority'], 4))
    else:
        print("Invalid sorting option!")
        return

    headers = ["#", "Task Name", "Due Date", "Priority", "Category"]
    table = [[i + 1, task['name'], task['due_date'], task['priority'], task['category']] for i, task in enumerate(sorted_tasks)]
    print(tabulate(table, headers=headers, tablefmt="grid"))

def filter_tasks_by_category(category):
    filtered_tasks = [task for task in tasks if task['category'].lower() == category.lower()]
    if not filtered_tasks:
        print(f"No tasks found under the category '{category}'.")
        return

    headers = ["#", "Task Name", "Due Date", "Priority", "Category"]
    table = [[i + 1, task['name'], task['due_date'], task['priority'], task['category']] for i, task in enumerate(filtered_tasks)]
    print(tabulate(table, headers=headers, tablefmt="grid"))

# Testing the setup
print("Task Management System Initialized!")
view_tasks()


Task Management System Initialized!
No tasks available.


In [12]:
import json

# Function to save tasks to a file
def save_tasks_to_file(filename="tasks.json"):
    try:
        with open(filename, "w") as file:
            json.dump(tasks, file, indent=4)
        print("Tasks saved successfully!")
    except Exception as e:
        print(f"Error saving tasks: {e}")

# Function to load tasks from a file
def load_tasks_from_file(filename="tasks.json"):
    global tasks
    try:
        with open(filename, "r") as file:
            tasks = json.load(file)
        print("Tasks loaded successfully!")
    except FileNotFoundError:
        print("No saved tasks found. Starting fresh.")
        tasks = []
    except Exception as e:
        print(f"Error loading tasks: {e}")

# Integrating history feature into task manager
def task_manager():
    load_tasks_from_file()

    while True:
        display_menu()
        choice = input("Enter your choice (1-15): ")

        if choice == '1':
            # Add a Task
            task_name = input("Enter the task name: ")
            due_date = input("Enter the due date (YYYY-MM-DD): ")
            priority = input("Enter the priority (High/Medium/Low): ")
            category = input("Enter the category (e.g., Work/Personal/Urgent): ")
            add_task(task_name, due_date, priority, category)
            save_tasks_to_file()

        elif choice == '2':
            # View All Tasks
            view_tasks()

        elif choice == '3':
            # Edit a Task
            view_tasks()
            try:
                index = int(input("Enter the task number to edit: ")) - 1
                new_name = input("Enter the new task name: ")
                new_due_date = input("Enter the new due date (YYYY-MM-DD): ")
                new_priority = input("Enter the new priority (High/Medium/Low): ")
                new_category = input("Enter the new category (e.g., Work/Personal/Urgent): ")
                edit_task(index, new_name, new_due_date, new_priority, new_category)
                save_tasks_to_file()
            except ValueError:
                print("Invalid input! Please enter a number for the task index.")

        elif choice == '4':
            # Delete a Task
            view_tasks()
            try:
                index = int(input("Enter the task number to delete: ")) - 1
                delete_task(index)
                save_tasks_to_file()
            except ValueError:
                print("Invalid input! Please enter a number for the task index.")

        elif choice == '6':
            # Sort Tasks by Due Date
            sort_tasks("due_date")

        elif choice == '7':
            # Sort Tasks by Priority
            sort_tasks("priority")

        elif choice == '8':
            # Filter Tasks by Category
            category = input("Enter the category to filter by: ")
            filter_tasks_by_category(category)

        elif choice == '9':
            # Mark Task as Completed
            view_tasks()
            try:
                index = int(input("Enter the task number to mark as completed: ")) - 1
                mark_task_completed(index)
                save_tasks_to_file()
            except ValueError:
                print("Invalid input! Please enter a number for the task index.")

        elif choice == '10':
            # Archive Completed Tasks
            archive_completed_tasks()

        elif choice == '11':
            # Check Due Dates
            check_due_dates()

        elif choice == '12':
            # Set Task Reminder
            view_tasks()
            try:
                index = int(input("Enter the task number to set a reminder: ")) - 1
                reminder_days_before = int(input("Enter the number of days before the due date to set the reminder: "))
                set_task_reminder(index, reminder_days_before)
                save_tasks_to_file()
            except ValueError:
                print("Invalid input! Please enter a number for the task index or reminder days.")

        elif choice == '13':
            # View Completed Tasks History
            view_completed_tasks_history()

        elif choice == '14':
            # Generate Task Analytics
            generate_task_analytics()

        elif choice == '15':
            # Exit
            print("Exiting Task Management System. Goodbye!")
            save_tasks_to_file()
            break
        else:
            print("Invalid choice! Please enter a number between 1 and 15.")


# Start the updated task manager
task_manager()

Tasks loaded successfully!

Task Management System
1. Add a Task
2. View All Tasks
3. Edit a Task
4. Delete a Task
5. Exit
6. Sort Tasks by Due Date
7. Sort Tasks by Priority
8. Filter Tasks by Category
9. Mark Task as Completed
10. Archive Completed Tasks
11. Check Due Dates (Reminder)
12. Set Task Reminder
13. View Completed Tasks History
14. Generate Task Analytics
15. Exit


Exiting Task Management System. Goodbye!
Tasks saved successfully!


# **GUI**

In [3]:
import tkinter as tk
from tkinter import simpledialog, messagebox

# Create the main window
root = tk.Tk()
root.title("Task Management System")
root.geometry("500x600")  # Adjust the window size to fit all buttons

# Placeholder task list
tasks = []

# Function that will be called when each button is clicked
def button_click(option):
    if option == 1:  # Add a Task
        add_task()
    elif option == 2:  # View All Tasks
        view_all_tasks()
    elif option == 3:  # Edit a Task
        edit_task()
    elif option == 4:  # Delete a Task
        delete_task()
    elif option == 5:  # Exit
        root.quit()
    elif option == 6:  # Sort Tasks by Due Date
        sort_tasks_by_due_date()
    elif option == 7:  # Sort Tasks by Priority
        sort_tasks_by_priority()
    elif option == 8:  # Filter Tasks by Category
        filter_tasks_by_category()
    elif option == 9:  # Mark Task as Completed
        mark_task_completed()
    elif option == 10:  # Archive Completed Tasks
        archive_completed_tasks()
    elif option == 11:  # Check Due Dates (Reminder)
        check_due_dates()
    elif option == 12:  # Set Task Reminder
        set_task_reminder()
    elif option == 13:  # View Completed Tasks History
        view_completed_tasks_history()
    elif option == 14:  # Generate Task Analytics
        generate_task_analytics()
    elif option == 15:  # Exit
        root.quit()

# Function to add a task
def add_task():
    task_name = simpledialog.askstring("Input", "Enter task name:")
    due_date = simpledialog.askstring("Input", "Enter due date (YYYY-MM-DD):")
    priority = simpledialog.askstring("Input", "Enter priority (High/Medium/Low):")
    category = simpledialog.askstring("Input", "Enter category:")
    task = {"name": task_name, "due_date": due_date, "priority": priority, "category": category, "completed": False}
    tasks.append(task)
    messagebox.showinfo("Task Added", f"Task '{task_name}' added successfully.")

# Function to view all tasks
def view_all_tasks():
    all_tasks = "\n".join([f"{task['name']} - Due: {task['due_date']} - Priority: {task['priority']} - Category: {task['category']}" for task in tasks])
    messagebox.showinfo("All Tasks", all_tasks if all_tasks else "No tasks available.")

# Function to edit a task
def edit_task():
    task_name = simpledialog.askstring("Input", "Enter the name of the task to edit:")
    for task in tasks:
        if task['name'] == task_name:
            new_name = simpledialog.askstring("Edit", f"Enter new name for task '{task_name}':")
            new_due_date = simpledialog.askstring("Edit", "Enter new due date (YYYY-MM-DD):")
            new_priority = simpledialog.askstring("Edit", "Enter new priority (High/Medium/Low):")
            new_category = simpledialog.askstring("Edit", "Enter new category:")
            task.update({"name": new_name, "due_date": new_due_date, "priority": new_priority, "category": new_category})
            messagebox.showinfo("Task Edited", f"Task '{task_name}' updated successfully.")
            return
    messagebox.showerror("Task Not Found", f"Task '{task_name}' not found.")

# Function to delete a task
def delete_task():
    task_name = simpledialog.askstring("Input", "Enter the name of the task to delete:")
    for task in tasks:
        if task['name'] == task_name:
            tasks.remove(task)
            messagebox.showinfo("Task Deleted", f"Task '{task_name}' deleted successfully.")
            return
    messagebox.showerror("Task Not Found", f"Task '{task_name}' not found.")

# Function to sort tasks by due date
def sort_tasks_by_due_date():
    tasks.sort(key=lambda x: x['due_date'])
    messagebox.showinfo("Tasks Sorted", "Tasks sorted by due date.")

# Function to sort tasks by priority
def sort_tasks_by_priority():
    priority_order = {"High": 1, "Medium": 2, "Low": 3}
    tasks.sort(key=lambda x: priority_order.get(x['priority'], 3))
    messagebox.showinfo("Tasks Sorted", "Tasks sorted by priority.")

# Function to filter tasks by category
def filter_tasks_by_category():
    category = simpledialog.askstring("Input", "Enter category to filter by:")
    filtered_tasks = [task for task in tasks if task['category'] == category]
    filtered_tasks_str = "\n".join([f"{task['name']} - Due: {task['due_date']} - Priority: {task['priority']}" for task in filtered_tasks])
    messagebox.showinfo("Filtered Tasks", filtered_tasks_str if filtered_tasks_str else "No tasks found in this category.")

# Function to mark a task as completed
def mark_task_completed():
    task_name = simpledialog.askstring("Input", "Enter the name of the task to mark as completed:")
    for task in tasks:
        if task['name'] == task_name:
            task['completed'] = True
            messagebox.showinfo("Task Completed", f"Task '{task_name}' marked as completed.")
            return
    messagebox.showerror("Task Not Found", f"Task '{task_name}' not found.")

# Function to archive completed tasks
def archive_completed_tasks():
    global tasks
    tasks = [task for task in tasks if not task['completed']]
    messagebox.showinfo("Archived Tasks", "Completed tasks archived successfully.")

# Function to check due dates (reminder)
def check_due_dates():
    today = "2024-12-25"  # Example: Get today's date (can use datetime module in real use case)
    due_tasks = [task for task in tasks if task['due_date'] == today]
    due_tasks_str = "\n".join([f"{task['name']} - Due: {task['due_date']}" for task in due_tasks])
    messagebox.showinfo("Due Tasks", due_tasks_str if due_tasks_str else "No tasks due today.")

# Function to set a task reminder
def set_task_reminder():
    task_name = simpledialog.askstring("Input", "Enter the task name to set reminder for:")
    reminder_time = simpledialog.askstring("Input", "Enter reminder time (HH:MM):")
    # You can implement reminder functionality here (e.g., send notification at the specified time)
    messagebox.showinfo("Reminder Set", f"Reminder for task '{task_name}' set at {reminder_time}.")

# Function to view completed tasks history
def view_completed_tasks_history():
    completed_tasks = [task for task in tasks if task['completed']]
    completed_tasks_str = "\n".join([f"{task['name']} - Completed on {task['due_date']}" for task in completed_tasks])
    messagebox.showinfo("Completed Tasks", completed_tasks_str if completed_tasks_str else "No completed tasks.")

# Function to generate task analytics
def generate_task_analytics():
    total_tasks = len(tasks)
    completed_tasks = len([task for task in tasks if task['completed']])
    pending_tasks = total_tasks - completed_tasks
    messagebox.showinfo("Task Analytics", f"Total Tasks: {total_tasks}\nCompleted Tasks: {completed_tasks}\nPending Tasks: {pending_tasks}")

# Create buttons for all 15 options
options = [
    "Add a Task", "View All Tasks", "Edit a Task", "Delete a Task", "Exit",
    "Sort Tasks by Due Date", "Sort Tasks by Priority", "Filter Tasks by Category",
    "Mark Task as Completed", "Archive Completed Tasks", "Check Due Dates (Reminder)",
    "Set Task Reminder", "View Completed Tasks History", "Generate Task Analytics", "Exit"
]

# Create buttons dynamically based on the options list
for i, option in enumerate(options, start=1):
    button = tk.Button(root, text=option, width=20, command=lambda i=i: button_click(i))
    button.grid(row=i, column=0, pady=5)

# Run the application
root.mainloop()
